In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tune the DreamBooth Stable Diffusion model using your own images

In [5]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

name, memory.total [MiB], memory.free [MiB]
Tesla P100-PCIE-16GB, 16384 MiB, 16280 MiB


## Install dependencies
Run in terminal
```
conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia
```

**NOTE: DID NOT WORK FOR REVIEWER** Switched to pip install instead

In [4]:
%%writefile requirements.txt
torch==1.9.1
torchvision
torchaudio
tensorboard==1.15.0
transformers
ftfy
bitsandbytes
gradio
ninja
nvidia-pyindex
nvidia-cuda-runtime
natsort

Overwriting requirements.txt


In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 172.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.16rc425 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.


In [9]:
!sed -i '151 i\            cuda_setup = CUDASetup.get_instance()' /opt/conda/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py

**HACK**: Manually uninstall the misplaced version of `nvidia_cublas_cu11`. Run the following command in the terminal.

```bash
pip uninstall nvidia_cublas_cu11
```

In [9]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
%pip install git+https://github.com/ShivamShrirao/diffusers
%pip install -U --pre triton
%pip install -U --pre xformers
#%pip install accelerate==0.12.0
%pip install accelerate==0.15.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/ShivamShrirao/diffusers to /var/tmp/pip-req-build-3yhnnchd
  Running command git clone --filter=blob:none --quiet https://github.com/ShivamShrirao/diffusers /var/tmp/pip-req-build-3yhnnchd
  Resolved https://github.com/ShivamShrirao/diffusers to commit c1f887eccd2127f81283403ebcf9e2e435c8a9da
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/1

## Configure paths for inputs and outputs

In [1]:
#@markdown Name/Path of the initial model.
MODEL_NAME = "CompVis/stable-diffusion-v1-4" #@param {type:"string"}

#@markdown Path for images of the concept for training.
INSTANCE_DIR = "/home/jupyter/DogInput" #@param {type:"string"}

# #@markdown A general name for class like dog for dog images.
CLASS_DIR = f"/home/jupyter/dog"

#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = False #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    # make sure google drive is unmounted before remount
    drive.mount('/content/drive')

#@markdown Enter the directory name to save model at.
OUTPUT_DIR = "stable_diffusion_weights/output" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/home/jupyter/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $INSTANCE_DIR
!mkdir -p $OUTPUT_DIR

#@markdown sks is a rare identifier, feel free to replace it.

[*] Weights will be saved at /home/jupyter/stable_diffusion_weights/output


## Start training

In [11]:
#@title Setting up all training args
from argparse import Namespace

args = Namespace(
    pretrained_model_name_or_path=MODEL_NAME,
    center_crop=True,
    instance_data_dir=INSTANCE_DIR,
    #@markdown `instance_prompt` is a prompt that should contain a good description of what your object or style is
    instance_prompt="dog"  #@param {type:"string"}
    ,
    #@markdown `class_prompt` is a prompt that should contain a good description of what the class your object belongs to
    class_prompt="dog"  #@param {type:"string"}
    ,
    resolution=512 #@param {type:"number"}
    ,
    learning_rate=5e-06  #@param {type:"number"}
    ,
    max_train_steps=1000  #@param {type:"number"}
    ,
    train_batch_size=1,
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
    mixed_precision="fp16",
    prior_loss_weight=1.0  #@param {type:"number"}
    ,
    sample_batch_size=4, # default: 2
    class_data_dir=CLASS_DIR,
    num_class_images=5  #@param {type:"number"}
    ,
    output_dir=OUTPUT_DIR,
    lr_scheduler="constant",
    lr_warmup_steps=0,
    seed=1337 #@param {type:"number"}
)

#@markdown ---

#@markdown #Memory Usage Tips

#@markdown Click `Show code`, add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

#@markdown remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

!accelerate launch train_dreambooth.py \
--pretrained_model_name_or_path=$MODEL_NAME \
--instance_data_dir=$INSTANCE_DIR \
--class_data_dir=$CLASS_DIR \
--output_dir=$OUTPUT_DIR \
--with_prior_preservation \
--prior_loss_weight=1.0 \
--instance_prompt="dog" \
--class_prompt="dog" \
--seed=1337 \
--resolution=512 \
--center_crop \
--train_batch_size=1 \
--mixed_precision="fp16" \
--use_8bit_adam \
--gradient_accumulation_steps=1 \
--learning_rate=5e-06  \
--lr_scheduler="constant" \
--lr_warmup_steps=0 \
--num_class_images=5 \
--sample_batch_size=4

[22:18:55] WARNING  The following values were not passed to       ]8;id=794748;file:///opt/conda/lib/python3.7/site-packages/accelerate/commands/launch.py\launch.py]8;;\:]8;id=834181;file:///opt/conda/lib/python3.7/site-packages/accelerate/commands/launch.py#1088\1088]8;;\
                    `accelerate launch` and had defaults used                   
                    instead:                                                    
                            `--num_processes` was set to a value                
                    of `1`                                                      
                            `--num_machines` was set to a value                 
                    of `1`                                                      
                            `--mixed_precision` was set to a                    
                    value of `'no'`                                             
                            `--dynamo_backend` was set to a value     

## View output of training

In [ ]:
#@markdown Specify the weights directory to use (leave blank for latest)
WEIGHTS_DIR = "" #@param {type:"string"}
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

[*] WEIGHTS_DIR=/home/jupyter/stable_diffusion_weights/output/5


In [16]:
!wget -q https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05/raw/2af0a9c0237ed98b863a75e1db21d7ed5541094f/convert_diffusers_to_sd.py

In [17]:
!mv {WEIGHTS_DIR}/* {OUTPUT_DIR}